# EOPF S3 OLCI L1 Product Data Structure Proposal


In [1]:
import os
import xarray as xr
import glob
from utils import display, get_ds_variables as get_ds
from EOProductDataStructure import EOProductBuilder, EOVariableBuilder, EOGroupBuilder


In [4]:
path_to_product = glob.glob("data/S3A_OL_1*.SEN3")[0]

# Groups definition
groups = {}
groups['coordinates'] = get_ds(path_to_product, names=["time_coordinates","geo_coordinates","tie_geo_coordinates"], process_tie=True)
groups['radiances'] = get_ds(path_to_product, names=["Oa%02d_radiance" %r for r in range(21)])
groups['tie_points'] = get_ds(path_to_product, names=["tie_meteo","tie_geometries"], process_tie=True)
groups['removed_pixels'] = get_ds(path_to_product, names=["removed_pixels"])
groups['instrument_data'] = get_ds(path_to_product, names=["instrument_data"])
groups['quality_flags'] = get_ds(path_to_product, names=["qualityFlags"])

# Create a new EOProduct instance
product_name = os.path.basename("S3_OLCI_L1")
product = EOProductBuilder("S3_OLCI_L1", coords=EOGroupBuilder('coords'))
# do the same work as before 
product.metadatas = ["xfdumanifest.xml"]


# global attributes
#get them from a netcdf file 
ds = xr.open_dataset(path_to_product + "/Oa01_radiance.nc",decode_times=False,mask_and_scale=False)
    
product.attrs = {att: f'{value[:50]} ...' if isinstance(value, str) else value for att, value in ds.attrs.items()}
    
# ==================== Product groups setting ========================

for group_name, ds in groups.items():
    group = EOGroupBuilder(group_name)
    group.attrs["description"] = f"{group_name} Data Group"
    group.dims = ds.dims

    for v, var in ds.variables.items():
        variable = EOVariableBuilder(v, default_attrs = False)
        variable.dtype = var.dtype
        variable.dims = var.dims
        variable.attrs = var.attrs
        group.variables.append(variable)

    product.groups.append(group)

product.attrs['metadata_files'] = '[xfdumanfist.xml]'

In [5]:
display(product.compute())


# 2. S3 SAR L1

In [6]:
sar_l1 = "data/S3A_SR_1_SRA____20200101T205454_20200101T214524_20200126T194646_3029_053_185______LN3_O_NT_003.SEN3"
ds = xr.open_dataset(sar_l1 + "/measurement.nc",decode_times=False,mask_and_scale=False)

FileNotFoundError: [Errno 2] No such file or directory: b'/home/cdubos_x/DPR/Notebooks/eopf_product_data_structure/data/S3A_SR_1_SRA____20200101T205454_20200101T214524_20200126T194646_3029_053_185______LN3_O_NT_003.SEN3/measurement.nc'

In [7]:
sar = EOProductBuilder("S3A_SR_1_SRA____", coords=EOGroupBuilder('coordinates'))

#global attributes
sar.attrs = {att: f'{value[:40]} ...' if isinstance(value, str) else value for att, value in ds.attrs.items()}

#Coordinates 
coords_names = []
for c in ds.coords:
    coord = ds.get(c)
    coords_names.append(c)
    coordinate = EOVariableBuilder(c)
    coordinate.dtype = coord.dtype
    coordinate.dims = coord.dims
    coordinate.attrs = coord.attrs
    
    for d in coordinate.dims:
        if d not in sar.coords.dims:
            sar.coords.dims.append(d)
    sar.coords.variables.append(coordinate)
    
#Groups
echo_lrm = EOGroupBuilder("echo_lrm")
echo_sar_ku = EOGroupBuilder("echo_sar_ku")
echo_plrm = EOGroupBuilder("echo_plrm")
acquisition = EOGroupBuilder("acquisition")


# Setup group variables
for v, var in ds.variables.items():
    if v not in coords_names: # exclude coordinates 
        variable = EOVariableBuilder(v.split('_echo')[0])
        variable.dtype = var.dtype
        variable.dims = var.dims
        variable.attrs = var.attrs

        try:
            ctes = ", ".join(c for c in var.coords)
            variable.attrs["coordinates"] = ctes
        except Exception:
            pass

        if "echo_lrm" in v:
            echo_lrm.variables.append(variable)
            for d in variable.dims:
                if d not in echo_lrm.dims:
                    echo_lrm.dims.append(d)
            
        elif "echo_sar_ku" in v:
            echo_sar_ku.variables.append(variable)

            for d in variable.dims:
                if d not in echo_sar_ku.dims:
                    echo_sar_ku.dims.append(d)
        else:
            echo_plrm.variables.append(variable)

            for d in variable.dims:
                if d not in echo_plrm.dims:
                    echo_plrm.dims.append(d)

sar.groups.append(echo_lrm)
sar.groups.append(echo_sar_ku)
sar.groups.append(echo_plrm)
sar.groups.append(acquisition)

#adding acquisition group
ds = xr.open_dataset(sar_l1 + "/acquisition.nc",decode_times=False,mask_and_scale=False)
coords_names = []
for c in ds.coords:
    coord = ds.get(c)
    coords_names.append(c)
    coordinate = EOVariableBuilder(c)
    coordinate.dtype = coord.dtype
    coordinate.dims = coord.dims
    coordinate.attrs = coord.attrs
        
    for d in coordinate.dims:
        if d not in sar.coords.dims:
            sar.coords.dims.append(d)

    sar.coords.variables.append(coordinate)
    
# Setup acquisition variables
for v in ds.variables:
    if v not in coords_names: # exclude coordinates 
        variable = EOVariableBuilder(v.split('_acq')[0])
        var = ds.get(v)
        variable.dtype = var.dtype
        variable.dims = var.dims
        variable.attrs = var.attrs
        
        try:
            ctes = ", ".join(c for c in var.coords)
            variable.attrs['coordinates'] = ctes
        except:
            pass
        
        acquisition.variables.append(variable)
        
        for d in variable.dims:
            if d not in acquisition.dims:
                acquisition.dims.append(d)


#adding metadata
sar.metadata = ["xfdumanifest.xml"]
sar.attributes['metadata_files'] = '[xfdumanifest.xml,]'
display(sar.compute())

FileNotFoundError: [Errno 2] No such file or directory: b'/home/cdubos_x/DPR/Notebooks/eopf_product_data_structure/data/S3A_SR_1_SRA____20200101T205454_20200101T214524_20200126T194646_3029_053_185______LN3_O_NT_003.SEN3/acquisition.nc'